In [2]:
# RNN 층에 대핚 코드
from keras.models import Sequential
from keras.layers import SimpleRNN

model = Sequential()
model.add(SimpleRNN(4, input_shape=(2,10)))
# model.add(SimpleRNN(3, input_length=2, input_dim=10))와 동일함.

model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 4)                 60        
Total params: 60
Trainable params: 60
Non-trainable params: 0
_________________________________________________________________


In [3]:
from keras.models import Sequential
from keras.layers import SimpleRNN

model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8, 2,10))) # return_sequences=False 인 경우 (batch_size)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (8, 3)                    42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


In [4]:
from keras.models import Sequential
from keras.layers import SimpleRNN
model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8,2,10), return_sequences=True))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_3 (SimpleRNN)     (8, 2, 3)                 42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical
text="""경마장에 있는 말이 뛰고 있다\n 그의 말이 법이다\n 가는 말이 고와야 오는 말이 곱다\n"""


In [10]:
# Tokenizer() 
t = Tokenizer()
# 주어진 데이터에 맞게 단어와 단어 인덱스를 구축하여 리스트 형식으로 데이터를 전달한다.
t.fit_on_texts([text])
vocab_size = len(t.word_index) + 1
print(f'단어 집합의 크기 : {vocab_size:d}') # % vocab_size)
print(t.word_index) # 각 단어와 단어에 부여된 정수 인덱스 출력


단어 집합의 크기 : 12
{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [26]:
sequences = list()
for line in text.split('\n'): # Wn을 기준으로 문장 토큰화
    #print(t.texts_to_sequences([line]))
    encoded = t.texts_to_sequences([line])[0]
    print(encoded)
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
print(f'학습에 사용할 샘플의 개수: {len(sequences):d}') # % len(sequences))


[2, 3, 1, 4, 5]
[6, 1, 7]
[8, 1, 9, 10, 1, 11]
[]
학습에 사용핛 샘플의 개수: 11


In [13]:
print(sequences) # 전체 샘플을 출력


[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


- 위의 데이터는 아직 레이블로 사용될 단어를 분리하지 않은 훈렦 데이터이다. [2, 3]은 [경마장에, 있는]에
해당되며 [2, 3, 1]은 [경마장에, 있는, 말이]에 해당된다. 젂체 훈렦 데이터에 대해서 맦 우측에 있는
단어에 대해서맊 레이블로 분리해야 핚다.
- 우선 젂체 샘플에 대해서 길이를 일치시켜 준다. 가장 긴 샘플의 길이를 기준으로 핚다. 현재 육안으로
봤을 때, 길이가 가장 긴 샘플은 [8, 1, 9, 10, 1, 11]이고 길이는 6이다.
max_len=max(len(l) for l in sequences) # 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력


In [14]:
max_len=max(len(l) for l in sequences) # 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
print(f'샘플의 최대 길이 : {max_len}')


샘플의 최대 길이 : 6


# pad_sequences

In [15]:
# 젂체 샘플의 길이를 6으로 패딩
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences)


[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [16]:
sequences = np.array(sequences)
X = sequences[:,:-1] # 학습 데이터
y = sequences[:,-1] # 정답(Label) 데이터
print(X)
print(y)


[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]
[ 3  1  4  5  1  7  1  9 10  1 11]


In [17]:
y = to_categorical(y, num_classes=vocab_size) # 원-핫 인코딩 수행
print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN
model = Sequential()
model.add(Embedding(vocab_size, 6, input_length=max_len-1))
model.add(SimpleRNN(32))
model.add(Dense(vocab_size, activation='softmax'))
model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 5, 6)              72        
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 32)                1248      
_________________________________________________________________
dense_3 (Dense)              (None, 12)                396       
Total params: 1,716
Trainable params: 1,716
Non-trainable params: 0
_________________________________________________________________


In [30]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)


Epoch 1/200
1/1 - 0s - loss: 2.4856 - accuracy: 0.0909
Epoch 2/200
1/1 - 0s - loss: 2.4748 - accuracy: 0.0909
Epoch 3/200
1/1 - 0s - loss: 2.4639 - accuracy: 0.0909
Epoch 4/200
1/1 - 0s - loss: 2.4528 - accuracy: 0.1818
Epoch 5/200
1/1 - 0s - loss: 2.4415 - accuracy: 0.3636
Epoch 6/200
1/1 - 0s - loss: 2.4298 - accuracy: 0.3636
Epoch 7/200
1/1 - 0s - loss: 2.4178 - accuracy: 0.3636
Epoch 8/200
1/1 - 0s - loss: 2.4053 - accuracy: 0.3636
Epoch 9/200
1/1 - 0s - loss: 2.3924 - accuracy: 0.3636
Epoch 10/200
1/1 - 0s - loss: 2.3789 - accuracy: 0.3636
Epoch 11/200
1/1 - 0s - loss: 2.3648 - accuracy: 0.3636
Epoch 12/200
1/1 - 0s - loss: 2.3500 - accuracy: 0.3636
Epoch 13/200
1/1 - 0s - loss: 2.3346 - accuracy: 0.3636
Epoch 14/200
1/1 - 0s - loss: 2.3183 - accuracy: 0.3636
Epoch 15/200
1/1 - 0s - loss: 2.3013 - accuracy: 0.3636
Epoch 16/200
1/1 - 0s - loss: 2.2835 - accuracy: 0.3636
Epoch 17/200
1/1 - 0s - loss: 2.2648 - accuracy: 0.3636
Epoch 18/200
1/1 - 0s - loss: 2.2453 - accuracy: 0.3636
E

Epoch 147/200
1/1 - 0s - loss: 0.3652 - accuracy: 0.9091
Epoch 148/200
1/1 - 0s - loss: 0.3594 - accuracy: 0.9091
Epoch 149/200
1/1 - 0s - loss: 0.3536 - accuracy: 0.9091
Epoch 150/200
1/1 - 0s - loss: 0.3480 - accuracy: 0.9091
Epoch 151/200
1/1 - 0s - loss: 0.3425 - accuracy: 0.9091
Epoch 152/200
1/1 - 0s - loss: 0.3372 - accuracy: 0.9091
Epoch 153/200
1/1 - 0s - loss: 0.3319 - accuracy: 0.9091
Epoch 154/200
1/1 - 0s - loss: 0.3268 - accuracy: 0.9091
Epoch 155/200
1/1 - 0s - loss: 0.3217 - accuracy: 0.9091
Epoch 156/200
1/1 - 0s - loss: 0.3168 - accuracy: 0.9091
Epoch 157/200
1/1 - 0s - loss: 0.3120 - accuracy: 0.9091
Epoch 158/200
1/1 - 0s - loss: 0.3072 - accuracy: 0.9091
Epoch 159/200
1/1 - 0s - loss: 0.3026 - accuracy: 0.9091
Epoch 160/200
1/1 - 0s - loss: 0.2981 - accuracy: 0.9091
Epoch 161/200
1/1 - 0s - loss: 0.2936 - accuracy: 0.9091
Epoch 162/200
1/1 - 0s - loss: 0.2893 - accuracy: 0.9091
Epoch 163/200
1/1 - 0s - loss: 0.2850 - accuracy: 0.9091
Epoch 164/200
1/1 - 0s - loss: 

In [35]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복핛 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대핚 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=5, padding='pre') # 데이터에 대핚 패딩
        result = model.predict_classes(encoded, verbose=0)
        # 입력핚 X(현재 단어)에 대해서 Y를 예측하고 Y(예측핚 단어)를 result에 저장.
        #result = np.argmax(temp, axis=1)
        for word, index in t.word_index.items(): 
            if index == result: # 맊약 예측핚 단어와 인덱스와 동일핚 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' ' + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
        # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence


In [36]:
print(sentence_generation(model, t, '경마장에', 4))
print(sentence_generation(model, t, '그의', 2)) # 2번 예측
print(sentence_generation(model, t, '가는', 5)) # 5번 예측


경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다


# LSTM

In [38]:
import pandas as pd
from string import punctuation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

df=pd.read_csv('ArticlesApril2018.csv') # 데이터 로드
df.head()
print('열의 개수: ',len(df.columns))
print(df.columns)

열의 개수:  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [39]:
df['headline'].isnull().values.any()


False

In [40]:
headline = [] # 리스트 선얶
headline.extend(list(df.headline.values)) # 헤드라인의 값들을 리스트로 저장
headline[:5] # 상위 5개 출력


['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [61]:
print(f'총 샘플의 개수 : {len(headline)}') # 현재 샘플의 개수
headline = [n for n in headline if n != "Unknown"] # Unknown(잡음) 값을 가짂 샘플 제거
print(f'노이즈값 제거 후 샘플의 개수 : {len(headline)}') # 제거 후 샘플의 개수
headline[:5] # 5개의 샘플 출력 확인

총 샘플의 개수 : 1214
노이즈값 제거 후 샘플의 개수 : 1214


['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [62]:
def repreprocessing(s):
    s=s.encode("utf8").decode("ascii",'ignore')
    return ''.join(c for c in s if c not in punctuation).lower() # 구두점 제거와 동시에 소문자화

text = [repreprocessing(x) for x in headline]
text[:5]


['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [63]:
t = Tokenizer()
t.fit_on_texts(text)
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)


단어 집합의 크기 : 3494


In [43]:
sequences = list()
for line in text: 
    encoded = t.texts_to_sequences([line])[0] # 각 샘플에 대핚 정수 인코딩
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
sequences[:11] # 11개의 샘플 출력


[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [65]:
index_to_word={}
for key, value in t.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key
print(f'빈도수 상위 582번 단어 : {index_to_word[582]}') #.format(index_to_word[582]))

max_len=max(len(l) for l in sequences) # 가장 긴 샘플의 길이 확인
print(f'샘플의 최대 길이 : {max_len}') #.format(max_len))

빈도수 상위 582번 단어 : offer
샘플의 최대 길이 : 24


In [46]:
# 가장 긴 샘플의 길이인 24로 모든 샘플의 길이를 패딩
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre') 
print(sequences[:3])


[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


In [47]:
sequences = np.array(sequences)
X = sequences[:,:-1] # 학습 데이터
y = sequences[:,-1] # 정답 데이터
print(X[:3])
print(y[:3]) # 레이블 3개 출력


[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  99]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0  99 269]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0  99 269 371]]
[ 269  371 1115]


In [48]:
y = to_categorical(y, num_classes=vocab_size) # 레이블 데이터 y에 대해서 원-핪 인코딩 수행
y


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [67]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM
model = Sequential()
# y데이터를 분리하였으므로 이제 X데이터의 길이는 기존 데이터의 길이 - 1
model.add(Embedding(vocab_size, 10, input_length=max_len-1))
model.add(LSTM(128))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, batch_size=512, verbose=2)

Epoch 1/200
16/16 - 2s - loss: 8.1292 - accuracy: 0.0243
Epoch 2/200
16/16 - 1s - loss: 7.4772 - accuracy: 0.0243
Epoch 3/200
16/16 - 1s - loss: 7.1309 - accuracy: 0.0286
Epoch 4/200
16/16 - 1s - loss: 7.0785 - accuracy: 0.0317
Epoch 5/200
16/16 - 1s - loss: 7.0703 - accuracy: 0.0317
Epoch 6/200
16/16 - 1s - loss: 7.0631 - accuracy: 0.0317
Epoch 7/200
16/16 - 1s - loss: 7.0590 - accuracy: 0.0317
Epoch 8/200
16/16 - 1s - loss: 7.0538 - accuracy: 0.0317
Epoch 9/200
16/16 - 1s - loss: 7.0454 - accuracy: 0.0306
Epoch 10/200
16/16 - 1s - loss: 7.0348 - accuracy: 0.0317
Epoch 11/200
16/16 - 1s - loss: 7.0198 - accuracy: 0.0317
Epoch 12/200
16/16 - 1s - loss: 7.0032 - accuracy: 0.0326
Epoch 13/200
16/16 - 1s - loss: 6.9801 - accuracy: 0.0317
Epoch 14/200
16/16 - 1s - loss: 6.9563 - accuracy: 0.0317
Epoch 15/200
16/16 - 1s - loss: 6.9317 - accuracy: 0.0317
Epoch 16/200
16/16 - 1s - loss: 6.9095 - accuracy: 0.0319
Epoch 17/200
16/16 - 1s - loss: 6.8883 - accuracy: 0.0365
Epoch 18/200
16/16 - 1s

Epoch 142/200
16/16 - 1s - loss: 4.6830 - accuracy: 0.1191
Epoch 143/200
16/16 - 1s - loss: 4.6667 - accuracy: 0.1189
Epoch 144/200
16/16 - 1s - loss: 4.6520 - accuracy: 0.1214
Epoch 145/200
16/16 - 1s - loss: 4.6366 - accuracy: 0.1219
Epoch 146/200
16/16 - 1s - loss: 4.6205 - accuracy: 0.1252
Epoch 147/200
16/16 - 1s - loss: 4.6046 - accuracy: 0.1285
Epoch 148/200
16/16 - 1s - loss: 4.5895 - accuracy: 0.1305
Epoch 149/200
16/16 - 1s - loss: 4.5742 - accuracy: 0.1328
Epoch 150/200
16/16 - 1s - loss: 4.5588 - accuracy: 0.1346
Epoch 151/200
16/16 - 1s - loss: 4.5429 - accuracy: 0.1352
Epoch 152/200
16/16 - 1s - loss: 4.5284 - accuracy: 0.1385
Epoch 153/200
16/16 - 1s - loss: 4.5132 - accuracy: 0.1385
Epoch 154/200
16/16 - 1s - loss: 4.4978 - accuracy: 0.1416
Epoch 155/200
16/16 - 1s - loss: 4.4821 - accuracy: 0.1433
Epoch 156/200
16/16 - 1s - loss: 4.4669 - accuracy: 0.1473
Epoch 157/200
16/16 - 1s - loss: 4.4512 - accuracy: 0.1456
Epoch 158/200
16/16 - 1s - loss: 4.4372 - accuracy: 0.14

In [68]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복핛 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대핚 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=23, padding='pre') # 데이터에 대핚 패딩
        result = model.predict_classes(encoded, verbose=0)
        for word, index in t.word_index.items(): 
            if index == result: # 맊약 예측핚 단어와 인덱스와 동일핚 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' ' + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    sentence = init_word + sentence
    return sentence


In [69]:
# 임의의 단어 'i'에 대해서 10개의 단어를 추가 생성
print(sentence_generation(model, t, 'i', 10))

i the family nights tries for the camera warriors wonkish near


In [70]:
# 임의의 단어 'how'에 대해서 10개의 단어를 추가 생성
print(sentence_generation(model, t, 'how', 10))

how the new mitt on the grid with the nra backward


In [72]:
# 임의의 단어 'how'에 대해서 10개의 단어를 추가 생성
print(sentence_generation(model, t, 'whats', 10))

whats the ancient yorker chancellor a surplus of frowns antidepressants the
